## Import libraries

In [1]:
import os 
import csv
import numpy as np
import pandas as pd
import random # library to generate random numbers
np.random.seed(seed=42)
import matplotlib.pyplot as plt # For plotting
import math
get_ipython().magic('matplotlib notebook')
from IPython.display import HTML
import warnings
warnings.filterwarnings('ignore')

#### Import data

In [2]:
data1 = pd.read_csv("cities_and_distances.csv",header=0)

data1

,Unnamed: 0,Bangalore,Jaipur,Varanasi,Udaipur,Delhi,Chennai,Mysore,Agra,Kolkata,Mumbai,Hyderabad
0,Bangalore,0,2026,1825,1724,2166,346,149,1923,1862,980,569
1,Jaipur,2026,0,849,393,287,2108,2121,240,1520,1146,1483
2,Varanasi,1825,849,0,1154,834,1862,1976,614,683,1524,1237
3,Udaipur,1724,393,1154,0,661,2077,1809,634,1825,753,1349
4,Delhi,2166,287,834,661,0,2203,2317,233,1498,1415,1578
5,Chennai,346,2108,1862,2077,2203,0,482,1957,1669,1345,627
6,Mysore,149,2121,1976,1809,2317,482,0,2072,2012,1064,718
7,Agra,1923,240,614,634,233,1957,2072,0,1280,1323,1334
8,Kolkata,1862,1520,683,1825,1498,1669,2012,1280,0,2053,1493
9,Mumbai,980,1146,1524,753,1415,1345,1064,1323,2053,0,711


#### Starting Point

In [3]:
starting_point = "Bangalore"

##### Format the data into required format

In [4]:
data1.reset_index(inplace=True)

data1  = data1.iloc[:,2:]

data1.index = data1.columns.values

data1

,Bangalore,Jaipur,Varanasi,Udaipur,Delhi,Chennai,Mysore,Agra,Kolkata,Mumbai,Hyderabad
Bangalore,0,2026,1825,1724,2166,346,149,1923,1862,980,569
Jaipur,2026,0,849,393,287,2108,2121,240,1520,1146,1483
Varanasi,1825,849,0,1154,834,1862,1976,614,683,1524,1237
Udaipur,1724,393,1154,0,661,2077,1809,634,1825,753,1349
Delhi,2166,287,834,661,0,2203,2317,233,1498,1415,1578
Chennai,346,2108,1862,2077,2203,0,482,1957,1669,1345,627
Mysore,149,2121,1976,1809,2317,482,0,2072,2012,1064,718
Agra,1923,240,614,634,233,1957,2072,0,1280,1323,1334
Kolkata,1862,1520,683,1825,1498,1669,2012,1280,0,2053,1493
Mumbai,980,1146,1524,753,1415,1345,1064,1323,2053,0,711


##### Map places to integers

###### Creating a mapping to the cities with integers so as to not clutter the code and to help generalize well

In [5]:
cities_mapping = {}

for i in enumerate(data1.columns):
    
    cities_mapping[i[0]] = i[1]

cities_mapping

{0: 'Bangalore',
 1: 'Jaipur',
 2: 'Varanasi',
 3: 'Udaipur',
 4: 'Delhi',
 5: 'Chennai',
 6: 'Mysore',
 7: 'Agra',
 8: 'Kolkata',
 9: 'Mumbai',
 10: 'Hyderabad'}

## getRoute:: use this function to generate random route

In [6]:
def getRoute():
    
    """ This function generates a route by sampling numbers """
    
    my_randoms = random.sample(range(1,11), 10)    # excluding 0 from the range as 0 is the starting point for us it is Bangalore
    
    return my_randoms

## fitnessFunction :: Function to compute the route cost using the already defined path 

Fitness function for us is the distance travelled if a particular path is taken.

    - Step 1: Use the generated route
    - Step 2: Calculate the distance travelled.

In [7]:
def fitnessFunction(pvRouteMap):
    
    """This functions takes the route generated and returns the cost incurred.
    Example :
        input : [8, 3, 5, 7, 1, 9, 10, 4, 6, 2]
        output : 17514
        
    Implementation:
        fitnessFunction([8, 3, 5, 7, 1, 9, 10, 4, 6, 2])
        output: 17514    
    """
    
    traverseData    = data1.copy() # creating a copy of the original df
    
    sourcePoint     = 0 # defining a starting point
    
    stopsCovered    = 0 # setting the number of stops covered to 0
    
    routeCost       = 0 # setting the initial cost to 0
    
    pvRouteMap1 = pvRouteMap #+ [pvRouteMap[0]]

    # initiate a while loop and calculate the cost for the whole path traversed and return the cost
    while(stopsCovered < len(pvRouteMap1)) :  
        
        routeCost = routeCost + traverseData.iloc[sourcePoint, pvRouteMap1[stopsCovered]]   
        """route cost is the sum of the cost incurred from travelling from one point to the next according
        to the route that was generated previously."""
        
        sourcePoint = pvRouteMap1[stopsCovered] # update the source point to the next point in the route map
        
        stopsCovered = stopsCovered + 1 # Adding 1 to the stops covered 
    
    routeCost = routeCost + traverseData.iloc[pvRouteMap1[-1],0] # for obtaining round trip distance

    return(routeCost)

In [8]:
print(fitnessFunction.__doc__)

This functions takes the route generated and returns the cost incurred.
    Example :
        input : [8, 3, 5, 7, 1, 9, 10, 4, 6, 2]
        output : 17514
        
    Implementation:
        fitnessFunction([8, 3, 5, 7, 1, 9, 10, 4, 6, 2])
        output: 17514    
    


#### We can look at the positions and think of one good route , we will see if our algorithm performs better than this route

In [9]:
one_route = [5, 6, 10, 9, 1, 3, 4, 7, 2, 8] # one of the better routes

In [10]:
one_route

[5, 6, 10, 9, 1, 3, 4, 7, 2, 8]

In [11]:
one_route_cities = []

for i in one_route:
    
    one_route_cities.append(cities_mapping[i])

one_route_cities = [starting_point] + one_route_cities + [starting_point]

one_route_cities

['Bangalore',
 'Chennai',
 'Mysore',
 'Hyderabad',
 'Mumbai',
 'Jaipur',
 'Udaipur',
 'Delhi',
 'Agra',
 'Varanasi',
 'Kolkata',
 'Bangalore']

In [12]:
fitnessFunction(one_route)

7849

##### Printing the actual route travelled with the city names

In [13]:
print("Total distance travelled to cover the final route of \n {} is {}".format(" => ".join( one_route_cities ),fitnessFunction(one_route)))

Total distance travelled to cover the final route of 
 Bangalore => Chennai => Mysore => Hyderabad => Mumbai => Jaipur => Udaipur => Delhi => Agra => Varanasi => Kolkata => Bangalore is 7849


###### We define a function to create inital population and calculate the cost for each solution

In [14]:
def initialPopCost(ninitpop = 10):
    
    """This function calculates the cost of the route that is passed.
    returns a dictionary of routes and the cost."""
    
    np.random.seed(seed=42)

    routeCost = [] 
    
    routes = [getRoute() for i in range(0, ninitpop)] # generating initial population's chromosomes
    
    routes_Cost = [fitnessFunction(i) for i in routes] # evaluating fitness of population
    
    routes_DF = pd.DataFrame([routes,routes_Cost]).transpose() # create dataframe with routes and cost
    
    routes_DF.columns = ['Route','Cost'] # Renaming columns
    
    return(routes_DF)

###### Example

In [15]:
initial_pop_cost = initialPopCost(100)

sorted_init_pop = initial_pop_cost.sort_values(['Cost'])

In [16]:
initial_pop_cost.head()

,Route,Cost
0,"[8, 2, 10, 7, 3, 9, 5, 1, 4, 6]",12709
1,"[10, 7, 9, 6, 1, 4, 3, 8, 2, 5]",12075
2,"[6, 1, 2, 9, 4, 8, 3, 5, 10, 7]",15342
3,"[2, 3, 7, 1, 4, 9, 8, 5, 6, 10]",11046
4,"[9, 8, 4, 10, 7, 1, 5, 6, 2, 3]",15127


In [17]:
sorted_init_pop.head()

,Route,Cost
3,"[2, 3, 7, 1, 4, 9, 8, 5, 6, 10]",11046
95,"[2, 3, 1, 7, 4, 10, 8, 5, 9, 6]",11143
65,"[6, 4, 7, 9, 3, 8, 2, 1, 5, 10]",11436
67,"[7, 8, 4, 2, 9, 1, 3, 10, 6, 5]",11493
93,"[9, 2, 1, 10, 6, 3, 4, 7, 8, 5]",11552


### Elitist approach

### Let us take top 5 (elite 5) solutions and do mutation on those

In [18]:
def theEliteFew(sorted_population, nelite = 5):
    
    """This functions picks 'nelite' number of best performing solutions.
    Takes input of population sorted based on the cost and returns the top 'nelite' """
    
    elite_few = sorted_population.head(nelite) # Selecting top nelite fittest parents
    
    return elite_few

In [19]:
elite_few_df = theEliteFew(sorted_init_pop)

In [20]:
elite_few_df

,Route,Cost
3,"[2, 3, 7, 1, 4, 9, 8, 5, 6, 10]",11046
95,"[2, 3, 1, 7, 4, 10, 8, 5, 9, 6]",11143
65,"[6, 4, 7, 9, 3, 8, 2, 1, 5, 10]",11436
67,"[7, 8, 4, 2, 9, 1, 3, 10, 6, 5]",11493
93,"[9, 2, 1, 10, 6, 3, 4, 7, 8, 5]",11552


##  getMutatedPath:: this function will generate mutated path where we define mutation policy.

### Mutation
    - Step 1: Pick a parent chromosome
    - Step 2: Randomly pick an index (position in the parent chromosome)
    - Step 3: Swap the values before and after the index to generate the mutated chromosome (child).

In [21]:
HTML('<img src="mutation.gif">')

In [22]:
def getMutatedPath(initPath, mutateFactor):
    
    """ This functions generates a mutated path , takes an input path and returns a mutated path.
    Mutate factor is the point at which the string (route here) is split and the two parts are swapped.
    
        Mutation
        Step 1: Pick a parent chromosome
        Step 2: Randomly pick an index (position in the parent chromosome)
        Step 3: Swap the values before and after the index to generate the mutated chromosome (child).
        
    Ex: 
        input to the function:        
            parent = [4, 6, 2, 8, 3, 5, 7, 1, 9, 10]
            index = 3 (randomy generate the index)
            
        with in function
            temp1 = [4, 6, 2]
            temp2 = [8, 3, 5, 7, 1, 9, 10]
        
        output:
            child  = [8, 3, 5, 7, 1, 9, 10, 4, 6, 2]

    Implemetation:
        getMutatedPath(initPath = [4, 6, 2, 8, 3, 5, 7, 1, 9, 10], mutateFactor = 3)
        output: [8, 3, 5, 7, 1, 9, 10, 4, 6, 2].
    """
    
    try:
        
        temp1 = initPath[0:mutateFactor] # Part 1 
        
        temp2 = initPath[mutateFactor:len(initPath)] # part 2
        
        newPath = temp2 + temp1 # Swapping Part2 + Part1
    
    except:
        
        # This loop is executed when you enter more than 1 value for mutate factor 
        
        temp1 = initPath[0:max(mutateFactor)] # Part 1
        
        temp2 = initPath[max(mutateFactor):] # part 2
        
        newPath = temp2 + temp1 # Swapping Part2 + Part1
    
    return(newPath)

In [23]:
print(getMutatedPath.__doc__)

 This functions generates a mutated path , takes an input path and returns a mutated path.
    Mutate factor is the point at which the string (route here) is split and the two parts are swapped.
    
        Mutation
        Step 1: Pick a parent chromosome
        Step 2: Randomly pick an index (position in the parent chromosome)
        Step 3: Swap the values before and after the index to generate the mutated chromosome (child).
        
    Ex: 
        input to the function:        
            parent = [4, 6, 2, 8, 3, 5, 7, 1, 9, 10]
            index = 3 (randomy generate the index)
            
        with in function
            temp1 = [4, 6, 2]
            temp2 = [8, 3, 5, 7, 1, 9, 10]
        
        output:
            child  = [8, 3, 5, 7, 1, 9, 10, 4, 6, 2]

    Implemetation:
        getMutatedPath(initPath = [4, 6, 2, 8, 3, 5, 7, 1, 9, 10], mutateFactor = 3)
        output: [8, 3, 5, 7, 1, 9, 10, 4, 6, 2].
    


#### Generate a sample path and mutate that path 

In [24]:
temp_route = getRoute()

In [25]:
temp_route

[2, 3, 6, 8, 9, 1, 5, 7, 10, 4]

In [26]:
getMutatedPath(temp_route,2)

[6, 8, 9, 1, 5, 7, 10, 4, 2, 3]

In [27]:
getMutatedPath(temp_route,2)[-1]

3

In [28]:
data1.iloc[getMutatedPath(temp_route,2)[-1],0] 

1724

In [29]:
getMutatedPath([4, 6, 2, 8, 3, 5, 7, 1, 9, 10],3)

[8, 3, 5, 7, 1, 9, 10, 4, 6, 2]

 # Mutation

In [30]:
temp = theEliteFew(sorted_init_pop)

In [31]:
pickedRouteMap = temp.sample(n=1)
pickedRouteMap

,Route,Cost
95,"[2, 3, 1, 7, 4, 10, 8, 5, 9, 6]",11143


In [32]:
pickedRouteMap.Route.tolist()[0]

[2, 3, 1, 7, 4, 10, 8, 5, 9, 6]

In [33]:
mutatedRouteMap = getMutatedPath(pickedRouteMap.Route.tolist()[0],2)

mutatedRouteMap

[1, 7, 4, 10, 8, 5, 9, 6, 2, 3]

In [34]:
getMutatedPath([4, 8, 2, 5, 10, 1, 3, 9, 0, 6, 7],2)

[2, 5, 10, 1, 3, 9, 0, 6, 7, 4, 8]

In [35]:
def mutationFunction(elite_few_df, percentage_to_mutate = 0.2):
    
    """This function mutates n input routes where n is calculated based on the percentage_to_mutate and returns the 
    corresponding solution generated and the cost."""
    
    # random index as mutate factor
    mutate_factor = random.choice(range(0,len(elite_few_df.Route.tolist()[1]),1))
    
    # selecting top 20% rows from the eliteDF 
    p = int(round(elite_few_df.shape[0]*percentage_to_mutate,0))
    
    # extracting routes of top 20%
    pickedRouteMaps = elite_few_df.Route.head(p).tolist()
    
    # passing the picked route maps to extract mutated child routes
    mutatedRoute_list = [getMutatedPath(i,mutate_factor) for i in pickedRouteMaps]
    
    # evaluating mutated child routes
    mutated_routes_Cost = [fitnessFunction(i) for i in mutatedRoute_list]
    
    # creating dataframe with routes and cost as column
    mutated_routes_DF = pd.DataFrame([mutatedRoute_list,mutated_routes_Cost]).transpose()
    
    # renaming column names
    mutated_routes_DF.columns = ['Route','Cost']
    
    return(mutated_routes_DF)

In [36]:
elite_few_df.Route.tolist()[1]

[2, 3, 1, 7, 4, 10, 8, 5, 9, 6]

In [37]:
elite_few_df.shape

(5, 2)

In [38]:
p = int(round(elite_few_df.shape[0] * 0.2, 0))

elite_few_df.Route.head(p).tolist()

[[2, 3, 7, 1, 4, 9, 8, 5, 6, 10]]

In [39]:
elite_few_df.head()

,Route,Cost
3,"[2, 3, 7, 1, 4, 9, 8, 5, 6, 10]",11046
95,"[2, 3, 1, 7, 4, 10, 8, 5, 9, 6]",11143
65,"[6, 4, 7, 9, 3, 8, 2, 1, 5, 10]",11436
67,"[7, 8, 4, 2, 9, 1, 3, 10, 6, 5]",11493
93,"[9, 2, 1, 10, 6, 3, 4, 7, 8, 5]",11552


In [40]:
mutationFunction(elite_few_df)

,Route,Cost
0,"[7, 1, 4, 9, 8, 5, 6, 10, 2, 3]",12902


## Partially Mapped Crossover

In [41]:
sorted_init_pop.head()

,Route,Cost
3,"[2, 3, 7, 1, 4, 9, 8, 5, 6, 10]",11046
95,"[2, 3, 1, 7, 4, 10, 8, 5, 9, 6]",11143
65,"[6, 4, 7, 9, 3, 8, 2, 1, 5, 10]",11436
67,"[7, 8, 4, 2, 9, 1, 3, 10, 6, 5]",11493
93,"[9, 2, 1, 10, 6, 3, 4, 7, 8, 5]",11552


###### - implemented PMX conceptualized by goldberg

## PMX Crossover

In [42]:
HTML('<img src = "pmx.gif">')

Taken from  - https://www.youtube.com/watch?v=c2ft8AG8JKE

In [43]:
def crossOverFunction(parent1, parent2):
    
    """This function implements the partially mapped crossover by goldeberg (https://www.hindawi.com/journals/cin/2017/7430125/)
    
    inputs: 2 parent solutions, crossover start position, crossover end position.
    crossover start position and  crossover end position are randomly generated in the solution.
    
    output: 2 child solutions, i.e the crossedover solutions.
    Example :
    
        Inputs:
            p1 = [5,7,2,4,8,9,3,6]
            p2 = [8,6,9,3,2,5,7,4]
            crossover_start_pos 2 (randomly selected)
            crossover_end_pos 6 (randomly selected)
        Internals:
            The mapping created is [(2, 9), (4, 3), (8, 2), (9, 5)]
            Child1 Intermediate [9, 7, 9, 3, 2, 5, 4, 6]
            Child1 Intermediate [2, 7, 9, 3, 2, 5, 4, 6]
            Child1 Intermediate [8, 7, 9, 3, 2, 5, 4, 6]
            Child1 final [8, 7, 9, 3, 2, 5, 4, 6]
            Child2 original [8, 6, 2, 4, 8, 9, 7, 4]
            Child2 Intermediate [2, 6, 2, 4, 8, 9, 7, 3]
            Child2 Intermediate [9, 6, 2, 4, 8, 9, 7, 3]
            Child2 Intermediate [5, 6, 2, 4, 8, 9, 7, 3]
            Child2 final [5, 6, 2, 4, 8, 9, 7, 3]
        
        Output:
            Child: [8, 7, 9, 3, 2, 5, 4, 6]
            Child: [5, 6, 2, 4, 8, 9, 7, 3].
    """
    
    # Generating two indices for crossover process
    indexes_for_crossover = random.sample(set(range(len(parent1))), 2)
    
    # selecting crossover starting and ending index
    crossover_factor_start_pos,crossover_factor_end_pos = min(indexes_for_crossover),max(indexes_for_crossover)
    
    ## generate child 1
    child1 = parent1[0:crossover_factor_start_pos]+\
             parent2[crossover_factor_start_pos:crossover_factor_end_pos] +\
             parent1[crossover_factor_end_pos:]
    
    ## generate child 2
    child2 = parent2[0:crossover_factor_start_pos] +\
             parent1[crossover_factor_start_pos:crossover_factor_end_pos] +\
             parent2[crossover_factor_end_pos:]
    
    ## Create mappings for the Genes with in those indices from two parents which underwent crossover
    mpping = list(zip(parent1[crossover_factor_start_pos:crossover_factor_end_pos],
                      parent2[crossover_factor_start_pos:crossover_factor_end_pos]))
    
    # run until all the nodes in the route are unique
    while len(np.unique(child1)) != len(child1):
        
        # extracting genes from child1 which did not undergo crossover
        child1_part = child1[:crossover_factor_start_pos]+child1[crossover_factor_end_pos:]
        
        for i in child1_part:
            
            for j in mpping:
                
                if i == j[1]:                  
                    
                    child1_part[child1_part.index(i)] = j[0]
                    
        child1 = child1_part[:crossover_factor_start_pos] + child1[crossover_factor_start_pos:crossover_factor_end_pos]+         child1_part[crossover_factor_start_pos:]

    # run until all the nodes in the route are unique
    while len(np.unique(child2)) != len(child2):
        
        # extracting genes from child2 which did not undergo crossover
        child2_part = child2[:crossover_factor_start_pos]+child2[crossover_factor_end_pos:]
        
        for i in child2_part:
            
            for j in mpping:
                
                if i == j[0]:
                    
                    child2_part[child2_part.index(i)] = j[1]
        
        child2 = child2_part[:crossover_factor_start_pos] + child2[crossover_factor_start_pos:crossover_factor_end_pos]+         child2_part[crossover_factor_start_pos:]
    
    return( child1,child2)

In [44]:
print(crossOverFunction.__doc__)

This function implements the partially mapped crossover by goldeberg (https://www.hindawi.com/journals/cin/2017/7430125/)
    
    inputs: 2 parent solutions, crossover start position, crossover end position.
    crossover start position and  crossover end position are randomly generated in the solution.
    
    output: 2 child solutions, i.e the crossedover solutions.
    Example :
    
        Inputs:
            p1 = [5,7,2,4,8,9,3,6]
            p2 = [8,6,9,3,2,5,7,4]
            crossover_start_pos 2 (randomly selected)
            crossover_end_pos 6 (randomly selected)
        Internals:
            The mapping created is [(2, 9), (4, 3), (8, 2), (9, 5)]
            Child1 Intermediate [9, 7, 9, 3, 2, 5, 4, 6]
            Child1 Intermediate [2, 7, 9, 3, 2, 5, 4, 6]
            Child1 Intermediate [8, 7, 9, 3, 2, 5, 4, 6]
            Child1 final [8, 7, 9, 3, 2, 5, 4, 6]
            Child2 original [8, 6, 2, 4, 8, 9, 7, 4]
            Child2 Intermediate [2, 6, 2, 4, 8, 9, 7, 3

In [45]:
def routesAfterCrossOver(sorted_pop, top_per = 0.2):
    
    """This functions takes in a population and performs crossover on the top_per records.
    output is a set of children after the crossover operation."""
    
    # selecting top_per% rows from the sorted_pop DF
    top_per = int(np.round((sorted_init_pop.shape[0] * top_per), 2))
    
    # extracting the top_per% of this sorted_pop DF
    top_few = sorted_pop.head(top_per)
    
    routes_crossover = [] 
    
    for i in range(int(top_few.shape[0])): # for every row randomly pick a pair to crossover
        
        # Randomly generate two row number in the top_few DF which means randomly selecting two population
        indexes = random.sample(top_few.index.tolist(),2)
        
        # Extracting Routes for the above two randomly picked rows(indexes) for crossover process
        temp1,temp2 = top_few.iloc[top_few.index==indexes[0]].Route.tolist()[0],top_few.iloc[top_few.index==indexes[1]].Route.tolist()[0]
        
        # Sending those two selected parents for crossover process
        sol1,sol2 = crossOverFunction(temp1,temp2)
        
        # Appending all the crossover childrens to the routes_crossover DF
        routes_crossover.extend([sol1,sol2])

    # Evaluating the fitness of the crossover childrens
    cost_crossover = [fitnessFunction(i) for i in routes_crossover]
    
    # creating dataframe with routes and cost of the child
    cross_over_DF = pd.DataFrame([routes_crossover,cost_crossover],).transpose()
    
    # Renaming the column names
    cross_over_DF.columns = ['Route','Cost']
    
    return cross_over_DF

In [46]:
routesAfterCrossOver(sorted_init_pop.head(6))

,Route,Cost
0,"[10, 4, 1, 7, 3, 8, 2, 5, 9, 6]",10236
1,"[6, 3, 7, 9, 4, 10, 8, 1, 5, 2]",15716
2,"[2, 4, 1, 7, 3, 10, 8, 5, 9, 6]",10889
3,"[5, 3, 4, 6, 9, 10, 8, 2, 7, 1]",12232
4,"[2, 3, 1, 7, 4, 10, 9, 5, 8, 6]",11309
5,"[8, 2, 1, 10, 6, 3, 4, 7, 9, 5]",11312
6,"[6, 3, 7, 1, 4, 9, 8, 5, 2, 10]",11924
7,"[5, 4, 7, 9, 3, 8, 2, 1, 6, 10]",11623
8,"[6, 4, 5, 9, 3, 1, 8, 2, 7, 10]",11880
9,"[7, 4, 3, 6, 9, 10, 2, 1, 5, 8]",14126


In [47]:
def overall_ga_run(noverall = 100 ,ninitpop=50):
    
    """This function runs the whole GA process, i.e
        1. Generate initial population
        2. Mutation
        3. Crossover
    and returns the best solution.
    
    Inputs : overall GA runs, initial population size.
    """
    
    ## create an empty dataframe to store the solutions
    all_solutions_generated = pd.DataFrame(columns=['Route','Cost'])
    
    #start a for loop and run the whole process for mentioned number of times
    """We only generate a population initially, after  the first run ,
        we take the best solutions from the previous run and continue with the process"""
    print("Starting {} iterations of the GA".format(noverall))

    for i in range(noverall):
        
        if all_solutions_generated.shape[0] == 0:
            
            initial_pop_cost = initialPopCost(ninitpop) # Generates initial population and cost associated with it
            
            sorted_init_pop = initial_pop_cost.sort_values('Cost') # sorting the population based on cost
        
        else:
            
            sorted_init_pop = all_solutions_generated.sort_values('Cost') # This is executed after 1st iteration
        
        # selecting the elite few
        elite_few_df = theEliteFew(sorted_init_pop,ninitpop)
        
        # Generating a random number based on which we either mutate or do a crossover
        matingFactor = np.random.uniform(0,1,1) # Random pick to decide on Mutation / crossover
        
        if matingFactor < 0.15:
            
            mutatedPopulationWthCost = mutationFunction(elite_few_df) # Returns mutated childrens DF
            
            all_solutions_generated.append(mutatedPopulationWthCost) # Append this to existing population
        
        else:
            
            crossoverPopulation = routesAfterCrossOver(elite_few_df) # Returns crossover DF
            
            all_solutions_generated = all_solutions_generated.append(crossoverPopulation) # Append this to existing population
        
        # drop duplicates does not compare lists, so we converted the routes to tuple
        all_solutions_generated.Route = all_solutions_generated.Route.map(lambda x : tuple(x))
        
        # Extracting unique solutions from all the solutions and are sorted to extract best solution after some iteration 
        unique_sols_generated  = all_solutions_generated.drop_duplicates().sort_values('Cost')
        
        # Converting back to list since all other functions are written by considering route to be list
        all_solutions_generated.Route = all_solutions_generated.Route.map(lambda x : list(x))
        
    print("Completed {} iterations of the GA".format(noverall))
    print("Total solutions generated till this point {}".format(all_solutions_generated.shape[0]),"\n")
    print ("-------------------------------------------------------------------------------------------" )
    return (all_solutions_generated,unique_sols_generated)

In [48]:
print(overall_ga_run.__doc__)

This function runs the whole GA process, i.e
        1. Generate initial population
        2. Mutation
        3. Crossover
    and returns the best solution.
    
    Inputs : overall GA runs, initial population size.
    


 ### Best Solution So Far


###### The number of possible solutions are: 10! = 3628800
We can generate all possible solutions and try to find the best of those , but this is a time taking process and may not be feasible for big problems.
Genetic algorithm approach takes a randominzed search approach to find the best solution, with enough number of iterations, mutation and cross over, GA can reach the global minima for most of the problems.

In [49]:
possible_solutions = math.factorial(10)

Total distance travelled to cover the final route of 
 Bangalore => Chennai => Hyderabad => Kolkata => Varanasi => Delhi => Agra => Jaipur => Udaipur => Mumbai => Mysore => Bangalore is 6815


In [50]:
overall_ga_run(1,ninitpop = 10)

Starting 1 iterations of the GA
Completed 1 iterations of the GA
Total solutions generated till this point 20 

-------------------------------------------------------------------------------------------


(                              Route   Cost
 0   [6, 4, 3, 8, 10, 1, 9, 2, 5, 7]  16340
 1   [10, 1, 3, 9, 5, 2, 7, 6, 4, 8]  14768
 2   [2, 10, 4, 5, 8, 1, 9, 7, 3, 6]  15093
 3   [1, 2, 6, 4, 8, 5, 10, 3, 9, 7]  16310
 4   [1, 6, 9, 10, 5, 2, 7, 8, 3, 4]  14957
 5   [1, 7, 9, 5, 2, 3, 4, 8, 6, 10]  13408
 6   [1, 5, 10, 9, 6, 4, 8, 2, 7, 3]  14006
 7   [9, 1, 3, 10, 5, 2, 4, 8, 6, 7]  14696
 8   [2, 6, 4, 5, 8, 1, 10, 3, 9, 7]  18341
 9   [1, 3, 9, 10, 5, 2, 4, 8, 7, 6]  12205
 10  [1, 4, 6, 7, 5, 10, 3, 8, 9, 2]  17862
 11  [3, 6, 9, 10, 5, 2, 4, 8, 7, 1]  13675
 12  [8, 4, 6, 3, 5, 2, 7, 10, 9, 1]  17256
 13  [6, 4, 7, 2, 5, 10, 3, 8, 1, 9]  12622
 14  [3, 4, 6, 7, 9, 10, 8, 2, 5, 1]  16980
 15  [10, 1, 2, 5, 6, 4, 3, 8, 9, 7]  15347
 16  [6, 7, 1, 10, 5, 2, 4, 9, 8, 3]  14284
 17  [1, 6, 9, 2, 5, 4, 7, 8, 10, 3]  16879
 18  [3, 9, 4, 8, 5, 2, 7, 10, 1, 6]  14622
 19  [2, 4, 8, 3, 1, 6, 7, 5, 10, 9]  14843,
                               Route   Cost
 9   (1, 3, 9, 10, 5, 2, 4, 8, 

In [51]:
from datetime import datetime

In [52]:
best_sol = []

for n in [10, 25, 50, 100, 150]: # pop size (initpopsize)
    
    for i in [10, 100]: # number of runs (noverall)
        
        initpopsize = n 
        
        runs = i
        
        start = datetime.now() 
        
        # Run GeneticAlgorithm for i iterations and for n initial Population size
        all_sols,unique_sols  = overall_ga_run(noverall = runs, ninitpop = initpopsize)
        
        # Extracting the first row from the unique_sols DF(which is sorted based on cost function hence fittest solution will be at the top)
        sol = list(unique_sols.iloc[0,:].values)
        
        print ("Completed {} runs of GA.".format(i))
        print ("{} Iterations of GA with and initial population size of {} took {} seconds".format(runs,initpopsize,(datetime.now()-start).total_seconds()))
        print ("-------------------------------------------------------------------------------------------" )
        print("Generated {}% of the {} solutions".format(np.round((len(all_sols)/possible_solutions)*100,3),possible_solutions))

    #### Final Solution for the combination of initpop and number of runs
    final_sol = unique_sols.head(1)

    #### Final Solution (Cities)
    final_route = []
    
    # This loop is for representing best solution in string format for every initial population value
    for i in final_sol.Route.values[0]:
        
        final_route.append(cities_mapping[i])
    
    final_route = [starting_point] + final_route + [starting_point]

    list(final_sol.Route.values[0])

    total_cost = fitnessFunction(list(final_sol.Route.values[0]))

    print("Total distance travelled to cover the final route of \n {} is {} KM. (Generated from initial population size of {} and runs {})".format(" => ".join(final_route),total_cost,initpopsize,runs))
    print ("-------------------------------------------------------------------------------------------" )
    
    best_sol.append(final_sol)

Starting 10 iterations of the GA
Completed 10 iterations of the GA
Total solutions generated till this point 180 

-------------------------------------------------------------------------------------------
Completed 10 runs of GA.
10 Iterations of GA with and initial population size of 10 took 1.196923 seconds
-------------------------------------------------------------------------------------------
Generated 0.005% of the 3628800 solutions
Starting 100 iterations of the GA
Completed 100 iterations of the GA
Total solutions generated till this point 1620 

-------------------------------------------------------------------------------------------
Completed 100 runs of GA.
100 Iterations of GA with and initial population size of 10 took 7.185546 seconds
-------------------------------------------------------------------------------------------
Generated 0.045% of the 3628800 solutions
Total distance travelled to cover the final route of 
 Bangalore => Hyderabad => Varanasi => Agra => 